# DEG Analysis using MAST
We will use the MAST R package to perform differential expression analysis on single-cell data. This is based on the best practices outlined [here](https://www.sc-best-practices.org/conditions/differential_gene_expression.html).

In [ ]:
input_file <- Sys.getenv("SNAKEMAKE_H5AD_INPUT")  # Input h5ad file
obs_group <- Sys.getenv("SNAKEMAKE_GROUP_OBS")  # Observations to group by
output_file <- Sys.getenv("SNAKEMAKE_OUTPUT_FILE")  # Output spreadsheet file
comparison <- Sys.getenv("SNAKEMAKE_COMPARISON")  # Comparison to perform, e.g., "condition1_VS_condition2&condition3
threads <- as.integer(Sys.getenv("SNAKEMAKE_NUM_THREADS", "1"))  # Number of threads to use
covariate_columns <- Sys.getenv("SNAKEMAKE_COVARIATES", "")  # Column in adata with sample information

# Check that the file exists
if (!file.exists(input_file)) {
  stop(paste("Input file does not exist:", input_file))
}

library(BiocParallel)
register(MulticoreParam(workers = threads), default = TRUE)
options(mc.cores = threads)  # Set the number of threads for parallel processing
Sys.setenv(OMP_NUM_THREADS = 1, MKL_NUM_THREADS = 1)

cat(paste("Input file:", input_file, "\n"))
cat(paste("Observations to group by:", obs_group, "\n"))
cat(paste("Output file:", output_file, "\n"))
cat(paste("Comparison:", comparison, "\n"))
cat(paste("Number of threads:", threads, "\n"))
cat(paste("Covariate columns:", covariate_columns, "\n"))

In [ ]:
# Load the input data as a SingleCellExperiment object
library(zellkonverter)
setZellkonverterVerbose(TRUE)
adata <- readH5AD(input_file)
adata

Prepare our data for MAST

In [ ]:
# Move the assay to the "counts" slot
library(SingleCellExperiment)
counts(adata) <- assay(adata, "X")
# We now need to normalize and log-transform the data
library(scuttle)
adata <- logNormCounts(adata, exprs_values = "counts", log = TRUE)

# Filter genes that are not expressed in at least 3 cells
expressed_in_cells <- rowSums(counts(adata) > 0)
adata <- adata[expressed_in_cells >= 3, ]

# Parse the comparisons and filter out cells not in either group

In [ ]:
get_group_list <- function(adata, comparison_string, exclude = NULL) {
    if (is.null(comparison_string) || comparison_string == "") {
        stop("Comparison string is empty or NULL.")
    }
    if (comparison_string == "rest") {
        # Select all groups except the excluded ones
        if (is.null(exclude)) {
            stop("Exclude parameter must be provided when comparison is 'rest'.")
        }
        groups <- unique(colData(adata)[[obs_group]])
        selected_groups <- setdiff(groups, exclude)
    } else {
        # Split the comparison string by "&" to get the groups
        selected_groups <- unlist(strsplit(comparison_string, "&"))
    }
    return(selected_groups)
}

# Get the groups for the comparison
split_comparison <- unlist(strsplit(comparison, "_VS_"))
ref_string <- split_comparison[1]
test_string <- split_comparison[2]

# Get the groups for the reference and test
ref_groups <- get_group_list(adata, ref_string)
test_groups <- get_group_list(adata, test_string, exclude = ref_groups)
cat("Using reference groups:", paste(ref_groups, collapse = ", "), "\n")
cat("Using test groups:", paste(test_groups, collapse = ", "), "\n")
# Filter the data to only include cells in the reference and test groups
if (length(ref_groups) == 0 || length(test_groups) == 0) {
    stop("No valid groups found for the comparison.")
}
adata <- adata[, colData(adata)[[obs_group]] %in% c(ref_groups, test_groups)]
adata

In [ ]:
# Create a new column in colData to indicate the group
colData(adata)$group <- ifelse(colData(adata)[[obs_group]] %in% ref_groups, "reference", "test")

Perform MAST analysis

In [ ]:
library(MAST)
# Create the MAST object
sca <- SceToSingleCellAssay(adata, class = "SingleCellAssay")
# Remove genes that are not expressed in at least 10% of the cells
sca <- sca[freq(sca) > 0.1, ]
cat("Filtered SCA shape:", dim(sca), "\n")
# Compute ngeneson
colData(sca)$ngeneson <- scale(colSums(assay(sca)>0))
# Convert group to a factor
colData(sca)$group <- factor(colData(sca)$group, levels = c("reference", "test"))
# Do the same for any covariates
covariate_columns <- unlist(strsplit(covariate_columns, ","))
for (covariate in covariate_columns) {
    if (covariate %in% colnames(colData(sca))) {
        colData(sca)[[covariate]] <- factor(colData(sca)[[covariate]])
    } else {
        warning(paste("Covariate column", covariate, "not found in colData. Skipping."))
    }
}
# Dynamically generate the formula for the model
# I.e. ~ group + ngeneson + (1 | covariate1) + (1 | covariate2) + ...
zlm_formula <- as.formula(paste("~ group + ngeneson",
                              paste(paste0("(1 | ", covariate_columns, ")"), collapse = " + ")))
# Generate the model
zlm_model <- zlm(
    formula = zlm_formula,
    sca=sca,
    method='glmer',
    ebayes=FALSE,
    strictConvergence=FALSE,
    fitArgsD=list(nAGQ=0),
    parallel=TRUE
)
# Select the ref vs test comparison with ref being the baseline
zlm_comparison <- summary(zlm_model, doLRT = "grouptest", parallel = TRUE)
# Extract the results
zlm_results <- zlm_comparison$datatable
# Reformat and filter to relevant columns
zlm_results <- merge(zlm_results[contrast=='grouptest' & component=='H',.(primerid, `Pr(>Chisq)`)], # p-values
                 zlm_results[contrast=='grouptest' & component=='logFC', .(primerid, coef, ci.hi, ci.lo)],
                 by='primerid') # logFC coefficients

# Convert logfc to log2 scale
zlm_results[,coef:=zlm_results[,coef]/log(2)]
# Perform multiple testing correction
zlm_results[,fdr:=p.adjust(`Pr(>Chisq)`, 'fdr')]
setorder(zlm_results, fdr)
# fixNAs that can sometimes be generated
fixNA = (is.na(zlm_results$coef) & zlm_results[, `Pr(>Chisq)`] < 1)
coefRange <- range(zlm_results$coef, na.rm = TRUE)
geneOrder = sapply(zlm_results$primerid, function(gene) {
    which(rownames(sca) == gene)
})
fracIn <- apply(assay(sca)[geneOrder, group == "ref"], 1, sum)/sum(sca$group == "ref")
fracOut <- apply(assay(sca)[geneOrder, group == "test"], 1, sum)/sum(sca$group == "test")
zlm_results[fixNA & facIn <= fracOut, "coef"] = coefRange[1]
zlm_results[fixNA & fracIn > fracOut, "coef"] = coefRange[2]
zlm_results

Export the results

In [ ]:
# Write the results to a file
write.csv(
    zlm_results,
    file = output_file,
)

Session Info

In [ ]:
sessionInfo()